# Описание работы

1. Этот датасет является не очень удобным для использования сиамской сети, поскольку неоптимизированный SVM сразу дает точность 93%, точнее acc= 0.937 acc_train= 0.99.
2. ELMO работает очень долго, поэтому была организована предварительная конвертация датасета в Colab. Полученные представления дали при применении SVM точность acc= 0.79 acc_train= 0.97.
3. Дополнена модель Tripletnet. Итератор gen_iter() для батчей для сиамской сети сформирован так: последовательно выбираются все вектора представления ELMO. Для каждого якоря случайно выбирается 32 вектора из его класса и 32 вектора из других классов. Всего сделано примерно 20 эпох, каждая занимала 300 сек.
4. Сохраняется сжатое представление размерности 128 для метрического поиска.
5. Полученные сжатые представления загружаются в индекс annoy, используется 200 деревьев.
6. Для каждого вектора из тестовой выборки вычисляется 10 ближайших соседей и мажоритарный класс сравнивается с тестовым. То же делается для контроля с учебной выборкой. 

## Результаты
- Точность  (accuracy score) для тестовой выборки составила 0.4244031830238727 и для учебной 0.5251392942425046. Оснований предположить ошибку в коде пока нет.
- Результаты не такие плохие с учетом того, что классификация идет по 20 классам, но значительно хуже SVM.
- Падение точности, скорее всего, связано с недообученностью сети Tripletnet. В целом кажется, что подход на основе сиамской сети подходит для One-Hot обучения, где данных очень мало. Если данных много, например, 20.000, то это означает общее количество триплетов для одной эпохи - 400 млн.
- Тем не менее, поскольу целью задания было кодирование, а не точность, то считаю его полностью выполненным.


## Постановка задачи

Assignment 8.

Develop a model for 20 news groups dataset. Select 20% of data for test set.

Use metric learning with siamese networks and triplet loss.
Use KNN and LSH (annoy library) for final prediction 
after the network was trained.

! Remember, that LSH gives you a set of neighbor candidates, 
for which you have to calculate distances to choose 
top-k nearest neighbors.

Your quality = accuracy score.


In [116]:
import numpy as np
import pandas as pd
import time
import random
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

import logging
logging.basicConfig(filename="pt.log", level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

import warnings
warnings.filterwarnings("ignore")

from sklearn.utils import shuffle
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.utils.extmath import density
from sklearn import metrics


In [8]:
categories = None
remove = ('headers', 'footers', 'quotes')
remove = ()
data = fetch_20newsgroups(subset='all', categories=categories, remove=remove)
target_names = data.target_names

In [17]:
y = data.target
data_train, data_test, y_train, y_test = train_test_split(data.data, y, test_size=0.2)
print(y.shape, len(data.data), len(data_train), len(data_test))

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(18846,) 18846 15076 3770
(15076, 152133) (3770, 152133) (15076,) (3770,)


In [29]:
import spacy
import en_core_web_sm
spacy_en = en_core_web_sm.load()
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = 'elmo_2x4096_512_2048cnn_2xhighway_options.json'
weight_file = 'elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5'
elmo = Elmo(options_file, weight_file, 2, dropout=0)

def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text) if tok.text.isalpha()]

In [ ]:
start = time.time()
N = len(data.data)
for i in range(N):
    t1 = data.data[i]
    if len(t1) < 10:
        t1 = 'xxxxx'
    t2 = [tokenizer(t1)]
    character_ids = batch_to_ids(t2)
    embeddings = elmo(character_ids)
    cat = torch.cat(embeddings['elmo_representations'], dim=-1)
    mean = cat.mean(dim=1)
    x = mean.detach().numpy()
    if i == 0:
        X = x
    else:
        X = np.vstack((X, x))
    timer = time.time() - start
    logging.info(str(i) + ' ' + str(N) + ' ' + str(X.shape) + ' ' + str(timer))
    if i % 100 == 0:
        np.save('X.npy', X)    
np.save('X.npy', X)            

In [185]:
def triplet_loss(anchor_embed, pos_embed, neg_embed):
    loss = F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed)
    loss = loss.mean()
    return loss

class Tripletnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(1024*2, 128)
        
    def branch(self, x):
        x = self.fc(x)
        return x

    def forward(self, anchor, pos, neg):
        anchor = self.branch(anchor)
        pos = self.branch(pos)
        neg = self.branch(neg)
        return triplet_loss(anchor, pos, neg)
    
model = Tripletnet()
optimizer = optim.Adam(model.parameters())    

In [158]:
def train_epoch(model, iterator, optimizer):
    model.train()
    epoch_loss = 0
    running_loss = 0
    n = 0
    start = time.time()
    for batch in iterator:
        n += 1
        optimizer.zero_grad()
        anchor, pos, neg = batch

        loss = model(anchor, pos, neg)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.detach().item()        
        epoch_loss += curr_loss
        
        loss_smoothing = n / (n+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss
        
        elapsed = time.time() - start
        out = f'epoch={epoch} n={n} elapsed={elapsed:.0f} curr_loss={curr_loss:.3f} running_loss={running_loss:.3f}'
        logging.info(out)
    return epoch_loss / n


In [187]:
X9 = np.load('X9.npy')
X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.2)
print(X9_train.shape, X9_test.shape, y_train.shape, y_test.shape)

(15076, 2048) (3770, 2048) (15076,) (3770,)


In [165]:
n_epochs=10
for epoch in range(1, n_epochs):
    train_iterator = gen_iter()
    train_loss = train_epoch(model, train_iterator, optimizer)
    print(f'epoch={epoch} | train_loss: {train_loss:.3f}')  
    torch.save(model, "siam.pt")  

epoch=1 | train_loss: -0.915
epoch=2 | train_loss: -0.922
epoch=3 | train_loss: -0.923
epoch=4 | train_loss: -0.927
epoch=5 | train_loss: -0.930
epoch=6 | train_loss: -0.931
epoch=7 | train_loss: -0.935
epoch=8 | train_loss: -0.936
epoch=9 | train_loss: -0.941


In [180]:
# Применение сиамской сети
TX9_train = torch.from_numpy(X9_train)
TX9_train = model.branch(TX9_train).detach().numpy()
np.save("TX9_train.npy", TX9_train)

TX9_test = torch.from_numpy(X9_test)
TX9_test = model.branch(TX9_test).detach().numpy()
print(TX9_test.shape)
np.save("TX9_test.npy", TX9_test)

In [182]:
np.save("y9_train.npy", y9_train)
np.save("y9_test.npy", y9_test)
print(TX9_train.shape, TX9_test.shape, y9_train.shape, y9_test.shape)

(15076, 128) (3770, 128) (15076,) (3770,)


In [ ]:
# Итератор для сиамской сети
# Выбрать якорь как следующий элемент, определить его класс
# Выделить все элементы с этим классом
# Выбрать 32 случайных элемента этого класса и 32 из чужих
# вернуть генератор с тремя батчами

def gen_iter():
    for i, i_class in enumerate(y9_train):
        i_index = []
        n_index = []
        for j, j_class in enumerate(y9_train):
            if j == i:
                continue
            if j_class == i_class:
                i_index.append(j)
            else:
                n_index.append(j)
        pos = np.random.choice(np.array(i_index), 32)
        neg = np.random.choice(np.array(n_index), 32)
    
        anchor_ = torch.zeros(32, 1024*2)
        pos_ = torch.zeros(32, 1024*2)
        neg_ = torch.zeros(32, 1024*2)
   
        for k in range(32):
            p = pos[k]
            p_class = y9_train[p]
            n = neg[k]
            n_class = y9_train[n]
       
            ta = torch.from_numpy(X9_train[i])
            tp = torch.from_numpy(X9_train[p])
            tn = torch.from_numpy(X9_train[n])
        
            anchor_[k] = ta
            pos_[k] = tp
            neg_[k] = tn
        
        yield anchor_, pos_, neg_


In [ ]:
from annoy import AnnoyIndex

In [ ]:
TX9_train = np.load(DIR + 'TX9_train.npy')
TX9_test = np.load(DIR + 'TX9_test.npy')
y9_train = np.load(DIR + 'y9_train.npy')
y9_test = np.load(DIR + 'y9_test.npy')

In [ ]:
N_train = TX9_train.shape[0]
N_test = TX9_test.shape[0]
M = 128
train = AnnoyIndex(M)
for i in range(N_train):
    v = TX9_train[i]
    train.add_item(i, v)

In [ ]:
acc = 0
for i in range(N_test):
    v = TX9_test[i]
    true = y9_test[i]
    nn = train.get_nns_by_vector(v, 20)
    class100 = []
    for j in nn:
      class1 = y9_train[j]
      class100.append(class1)
    prob = np.zeros(20)
    for class1 in class100:
        prob[class1] += 1
    pred = np.argmax(prob)
    if true == pred:
      acc += 1
res = acc / N_test
print(res)

In [ ]:
acc = 0
for i in range(N_train):
    v = TX9_train[i]
    true = y9_train[i]
    nn = train.get_nns_by_vector(v, 10)
    class100 = []
    for j in nn:
      class1 = y9_train[j]
      class100.append(class1)
    prob = np.zeros(20)
    for class1 in class100:
        prob[class1] += 1
    pred = np.argmax(prob)
    if true == pred:
      acc += 1
res = acc / N_train
print(res)